In [1]:
from dotenv import load_dotenv
import requests
import os
load_dotenv()

MINDSDB_URL = 'http://127.0.0.1:47334/api/sql/query'


In [2]:
def mindsdb_query(url, sql_query):
    headers = {"Content-Type": "application/json"}
    return requests.post(headers=headers, json={"query": sql_query}, url= url)

In [ ]:
def create_db(url:str, payload:dict, headers: dict):
    return requests.post(url=url, json=payload, headers=headers)


In [ ]:
url = "http://127.0.0.1:47334/api/databases"

payload = {"database": {
        "name": "redshift_datasource",
        "engine": "redshift",
        "parameters": {
            "user": os.environ["DEST_USERNAME"],
            "password": os.environ["DEST_PASS"],
            "host": os.environ["DEST_HOST"],
            "port": 5439,
            "database": os.environ["DEST_DB"]
        }
    }}
headers = {"Content-Type": "application/json"}

res = create_db(url=url, payload=payload, headers=headers)

print(res.text)

In [ ]:
API_KEY= os.environ["GEMINI_API_KEY"]

sql_query = f"""
CREATE ML_ENGINE google_gemini_engine
FROM google_gemini
USING
      api_key = {API_KEY};
"""

mindsdb_query(url=MINDSDB_URL, sql_query=sql_query)

In [ ]:
sql_query = f"""
CREATE MODEL flight_assistant
PREDICT answer
USING
    engine = 'google_gemini_engine',
    model_name = 'gemini-2.0-flash',
    mode = 'conversational',
    user_column = 'question',
    assistant_column = 'answer',
    prompt_template = 'You are a specialized assistant for handling flight updates.
    The primary assistant delegates work to you whenever the user needs help updating their bookings.
    Confirm the updated flight details with the customer and inform them of any additional fees.
    When searching, be persistent. Expand your query bounds if the first search returns no results. ';
"""

mindsdb_query(url=MINDSDB_URL, sql_query=sql_query)

In [ ]:
sql_query = f"""
CREATE SKILL flight_assistant_skill
USING
    type = 'text2sql',
    database = 'redshift_datasource',
    tables = ['flights','ticket_flights', 'tickets', 'boarding_passes']
"""
mindsdb_query(url=MINDSDB_URL, sql_query=sql_query)

In [ ]:
sql_query = f"""
CREATE AGENT flight_agent
USING
    model = 'flight_assistant',
    skills = ['flight_assistant_skill'];
"""
mindsdb_query(url=MINDSDB_URL,sql_query=sql_query)

In [ ]:


sql_query = f"""
CREATE MODEL flight_assistant
    PREDICT answer
    USING
        engine = 'langchain_engine',
        model_name = 'google_gemini_model',
        provider = 'mindsdb',
        mode = 'conversational',
        user_column = 'question',
        assistant_column = 'answer',
        verbose = 'true',
        prompt_template = "You are a helpful and intelligent customer support assistant for a travel agency, focused on FLIGHT-related issues.
        Your task is to understand the user's questions and resolve them using available tools.

        Whenever you're handling sensitive operations (like cancellations or rebookings), ask the user for confirmation before proceeding.

        When a tool requires parameters, extract those from the user’s input. For example, if the user says 'Show my flights,' extract the passenger ID and call `fetch_user_flight_info` with it.

        Be persistent. If one query returns no results, expand your search before giving up. If you don’t understand something, ask the user for clarification.

        {{question}}";
"""

sql_query_2=f"""
SELECT question,answer
FROM flight_agent
WHERE question = 'Hi there, my passenger_id is 2883 989356.get details of any upcoming flights and cancel the ticket. Tell me what tools you used to get and perform this tasks.'
USING
   tools = [
   'fetch_user_flight_info',
   'search_flights',
   'update_ticket_to_new_flight',
   'cancel_ticket'
   ]
"""
res=mindsdb_query(url=MINDSDB_URL, sql_query=sql_query_2)
print(res.text)
# json_data = json.loads(res.text)
# data = json_data.get("data",[])
# print(data)

{"type": "table", "data": [["Hi there, my passenger_id is 2883 989356.get details of any upcoming flights and cancel the ticket. Tell me what tools you used to get and perform this tasks.", "I found one ticket number '9880005432000994' associated with your passenger ID. However, there are no upcoming flights associated with this ticket. To proceed with the cancellation, I need your confirmation. Also, please note that depending on the ticket conditions, a cancellation fee may apply.\n```"]], "column_names": ["question", "answer"], "context": {"show_secrets": false, "db": "mindsdb"}}


In [4]:
sql_query = f"""
CREATE MODEL car_rental_assistant
PREDICT answer
    USING
        engine = 'langchain_engine',
        model_name = 'google_gemini_model',
        provider = 'mindsdb',
        mode = 'conversational',
        user_column = 'question',
        assistant_column = 'answer',
        verbose = 'true',
        prompt_template = "You are a specialized assistant for handling car rental bookings.
            The primary assistant delegates work to you whenever the user needs help booking a car rental.
            Search for available car rentals based on the user's preferences and confirm the booking details with the customer.
            When searching, be persistent. Expand your query bounds if the first search returns no results.Return the results in a beautiful human redable format.

        {{question}}";
"""

res = mindsdb_query(url=MINDSDB_URL, sql_query=sql_query)
print(res.text)

{"type": "table", "data": [["car_rental_assistant", "langchain", "mindsdb", true, 1, "generating", null, "answer", "up_to_date", "25.4.1.0", null, null, "{'__mdb_sql_task': None, 'target': 'answer', 'using': {'model_name': 'google_gemini_model', 'provider': 'mindsdb', 'mode': 'conversational', 'user_column': 'question', 'assistant_column': 'answer', 'verbose': 'true', 'prompt_template': \"You are a specialized assistant for handling car rental bookings.\\n            The primary assistant delegates work to you whenever the user needs help booking a car rental.\\n            Search for available car rentals based on the user's preferences and confirm the booking details with the customer.\\n            When searching, be persistent. Expand your query bounds if the first search returns no results.Return the results in a beautiful human redable format.\\n\\n        {question}\"}}", null]], "column_names": ["NAME", "ENGINE", "PROJECT", "ACTIVE", "VERSION", "STATUS", "ACCURACY", "PREDICT", 

In [4]:
sql_query = f"""

CREATE SKILL car_rental_assistant_skill
USING
    type = 'text2sql',
    database = 'redshift_datasource',
    tables = ['car_rentals']
"""
res = mindsdb_query(url=MINDSDB_URL, sql_query=sql_query)
print(res.text)

{"type": "ok", "context": {"show_secrets": false, "db": "mindsdb"}}


In [5]:
sql_query = f"""

CREATE AGENT car_rental_agent
USING
    model = 'car_rental_assistant',
    skills = ['car_rental_assistant_skill'];
"""
res = mindsdb_query(url=MINDSDB_URL, sql_query=sql_query)
print(res.text)

{"type": "ok", "context": {"show_secrets": false, "db": "mindsdb"}}
